# OpenBB Agents
This is a short example notebook to demonstrate some of the features of `openbb-agents`, and to get you up and running as quickly as possible.

## Setup
We'll be fetching our data provider API keys using OpenBB Platform's backend for this example (it's free to sign up!).

You are also free to set-up your [API keys](https://my.openbb.co/app/platform/pat) locally, or to rely only on `yfinance`
as a provider for data provider (although keep in mind that certain functions
will not be available as a result).

In [23]:
import os
import json
import random

import pandas as pd
from openbb import obb
from openbb_agents.agent import openbb_agent

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
OPENBB_PAT = os.environ.get('OPENBB_PAT')

### Basic Querying

In [7]:
result = openbb_agent("What is the current stock price of TSLA?", openbb_pat=OPENBB_PAT)


2024-07-15 22:23:18,603 - INFO - openbb_agents.agent - Using all available OpenBB tools with providers: ['yfinance']
2024-07-15 22:23:18,604 - INFO - openbb_agents.tools - Building OpenBB tool vector index...
2024-07-15 22:23:29,829 - INFO - openbb_agents.agent - Generating subquestions for user query: What is the current stock price of TSLA?
2024-07-15 22:23:31,109 - INFO - openbb_agents.agent - Generated subquestions: [SubQuestion(id=1, question='What is the current stock price of TSLA?', depends_on=None)]
2024-07-15 22:23:31,110 - INFO - openbb_agents.agent - Answering subquestion: id=1 question='What is the current stock price of TSLA?' depends_on=None
2024-07-15 22:23:31,111 - INFO - openbb_agents.agent - Attempting to select tools for: {'What is the current stock price of TSLA?'}
2024-07-15 22:23:32,034 - INFO - openbb_agents.chains - Searching tool index for: stock price
2024-07-15 22:23:33,101 - INFO - openbb_agents.agent - Retrieved tool(s): ['quote', 'historical']
2024-07-15 

In [8]:
# You can turn off verbose output to only receive the answer.
result = openbb_agent("What is the current stock price of AAPL?", verbose=False)
print(result)

- The current stock price of Apple Inc. (AAPL) is $234.40.
- This data is retrieved from the latest available quote.


### Use only specific OpenBB Platform tools
You don't have to rely on the entire suite of OpenBB tools. In certain
use-cases, it makes sense to limit the agent to only a small subset of the
tools.

In [24]:
# For example, if we only wanted our agent to use the `.equity.fundamental.income` tool:
data = obb.equity.fundamental.income("MSFT")

# Parse JSON data
data_list = json.loads(data)

# Convert to DataFrame
df = pd.DataFrame(data_list)

# Display DataFrame
print(df)

         period_ending  operating_revenue  total_revenue  cost_of_revenue  \
0  2023-06-30T00:00:00       2.119150e+11   2.119150e+11     6.586300e+10   
1  2022-06-30T00:00:00       1.982700e+11   1.982700e+11     6.265000e+10   
2  2021-06-30T00:00:00       1.680880e+11   1.680880e+11     5.223200e+10   
3  2020-06-30T00:00:00       1.430150e+11   1.430150e+11     4.607800e+10   

   gross_profit  other_gand_a  general_and_administrative_expense  \
0  1.460520e+11  7.575000e+09                        7.575000e+09   
1  1.356200e+11  5.900000e+09                        5.900000e+09   
2  1.158560e+11  5.107000e+09                        5.107000e+09   
3  9.693700e+10  5.111000e+09                        5.111000e+09   

   selling_and_marketing_expense  selling_general_and_admin_expense  \
0                   2.275900e+10                       3.033400e+10   
1                   2.182500e+10                       2.772500e+10   
2                   2.011700e+10                       

/home/anubrata/miniconda3/envs/testingopenai/lib/python3.10/site-packages/openbb_yfinance/models/income_statement.py:82: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna("N/A").replace("N/A", None).to_dict()


In [12]:
#... then we can specify it using the `openbb_tools` input argument
result = openbb_agent("What is the most recent annual revenue of MSFT?", openbb_tools=[".equity.fundamental.income"])
print(result)

2024-07-15 22:28:16,635 - INFO - openbb_agents.agent - Using specified OpenBB tools: ['.equity.fundamental.income']
2024-07-15 22:28:18,349 - INFO - openbb_agents.agent - Generating subquestions for user query: What is the most recent annual revenue of MSFT?
2024-07-15 22:28:19,199 - INFO - openbb_agents.agent - Generated subquestions: [SubQuestion(id=1, question='What is the most recent annual revenue of MSFT?', depends_on=None)]
2024-07-15 22:28:19,200 - INFO - openbb_agents.agent - Answering subquestion: id=1 question='What is the most recent annual revenue of MSFT?' depends_on=None
2024-07-15 22:28:19,201 - INFO - openbb_agents.agent - Attempting to select tools for: {'What is the most recent annual revenue of MSFT?'}
2024-07-15 22:28:20,149 - INFO - openbb_agents.chains - Searching tool index for: annual revenue
2024-07-15 22:28:21,396 - INFO - openbb_agents.agent - Retrieved tool(s): ['income']
2024-07-15 22:28:21,397 - INFO - openbb_agents.agent - Answering subquestion: What is 

/home/anubrata/miniconda3/envs/testingopenai/lib/python3.10/site-packages/openbb_yfinance/models/income_statement.py:82: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna("N/A").replace("N/A", None).to_dict()


2024-07-15 22:28:24,718 - INFO - openbb_agents.agent - Answered subquestion: - The most recent annual revenue of Microsoft Corporation (MSFT) is $211.915 billion for the fiscal year ending on June 30, 2023.
- This data is based on the latest available income statement for the period ending June 30, 2023.
- The most recent annual revenue of Microsoft Corporation (MSFT) is $211.915 billion for the fiscal year ending on June 30, 2023.
- This data is based on the latest available income statement for the period ending June 30, 2023.


### Using custom tools
You're also not limited to OpenBB's suite of tools. If you have custom tools you'd like to specify and add to the tool index (so that the LLM can search for your tool while c ompleting tasks), you can do so simply by adding a custom tool to the agent.


In [16]:
# def get_weather(location: str) -> float:
#     """Get the weather for a given location, in degress Celsius."""
#     return random.uniform(0, 50)

# # We can make extra tools available to the agent using the `extra_tools` input argument
# result = openbb_agent("What is the weather at AMZN's headquarters?", extra_tools=[get_weather])
# print(result)

### Async agent
An `async` agent is also available for use in your asnychronous workflows. This can lead to speed-ups in certain situations (we're still improving this over time!).

In [18]:
openbb_agent("What is the stock price of AAPL and MSFT?")

2024-07-15 22:33:02,548 - INFO - openbb_agents.agent - Using all available OpenBB tools with providers: ['yfinance']
2024-07-15 22:33:02,548 - INFO - openbb_agents.tools - Building OpenBB tool vector index...
2024-07-15 22:33:12,922 - INFO - openbb_agents.agent - Generating subquestions for user query: What is the stock price of AAPL and MSFT?
2024-07-15 22:33:14,506 - INFO - openbb_agents.agent - Generated subquestions: [SubQuestion(id=1, question='What is the current stock price of AAPL?', depends_on=[]), SubQuestion(id=2, question='What is the current stock price of MSFT?', depends_on=[])]
2024-07-15 22:33:14,507 - INFO - openbb_agents.agent - Answering subquestion: id=1 question='What is the current stock price of AAPL?' depends_on=[]
2024-07-15 22:33:14,508 - INFO - openbb_agents.agent - Attempting to select tools for: {'What is the current stock price of AAPL?'}
2024-07-15 22:33:16,151 - INFO - openbb_agents.chains - Searching tool index for: stock price
2024-07-15 22:33:17,425 -

'- The current stock price of Apple Inc. (AAPL) is $234.40.\n- The current stock price of Microsoft Corporation (MSFT) is $453.96.'

In [21]:
# openbb_agent("Perform a fundamentals financial analysis of AMZN using the most recently available data. What do you find that's interesting?")

In [20]:
openbb_agent("Who are TSLA's peers? What is their respective market cap? Return the results in _descending_ order of market cap.")

2024-07-15 22:36:47,398 - INFO - openbb_agents.agent - Using all available OpenBB tools with providers: ['yfinance']
2024-07-15 22:36:47,399 - INFO - openbb_agents.tools - Building OpenBB tool vector index...
2024-07-15 22:36:58,800 - INFO - openbb_agents.agent - Generating subquestions for user query: Who are TSLA's peers? What is their respective market cap? Return the results in _descending_ order of market cap.
2024-07-15 22:37:00,012 - INFO - openbb_agents.agent - Generated subquestions: [SubQuestion(id=1, question="Who are TSLA's peers?", depends_on=None), SubQuestion(id=2, question="What is the market cap of each of TSLA's peers?", depends_on=[1])]
2024-07-15 22:37:00,013 - INFO - openbb_agents.agent - Answering subquestion: id=1 question="Who are TSLA's peers?" depends_on=None
2024-07-15 22:37:00,014 - INFO - openbb_agents.agent - Attempting to select tools for: {"Who are TSLA's peers?"}
2024-07-15 22:37:00,828 - INFO - openbb_agents.chains - Searching tool index for: peers
202

/home/anubrata/miniconda3/envs/testingopenai/lib/python3.10/site-packages/pydantic/main.py:398: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


2024-07-15 22:37:28,476 - INFO - openbb_agents.chains - Function call: metrics({'symbol': 'F', 'period': 'annual'})
2024-07-15 22:37:28,549 - INFO - openbb_agents.chains - Function call: metrics({'symbol': 'GM', 'period': 'annual'})
2024-07-15 22:37:28,622 - INFO - openbb_agents.chains - Function call: metrics({'symbol': 'RIVN', 'period': 'annual'})
2024-07-15 22:37:28,694 - INFO - openbb_agents.chains - Function call: metrics({'symbol': 'NIO', 'period': 'annual'})


/home/anubrata/miniconda3/envs/testingopenai/lib/python3.10/site-packages/pydantic/main.py:398: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


2024-07-15 22:37:32,030 - INFO - openbb_agents.agent - Answered subquestion: Based on the retrieved data, here are the market capitalizations of Tesla's (TSLA) peers in descending order:

- **Ford Motor Company (F)**
  - Market Cap: $56.54 billion
  - Last Price: $14.23

- **General Motors Company (GM)**
  - Market Cap: $56.22 billion
  - Last Price: $49.30

- **Rivian Automotive, Inc. (RIVN)**
  - Market Cap: $17.42 billion
  - Last Price: $17.50

- **NIO Inc. (NIO)**
  - Market Cap: $9.66 billion
  - Last Price: $4.63

These companies are all involved in the design, manufacture, and sale of vehicles, making them direct competitors or peers to Tesla in the automotive industry.


"Based on the retrieved data, here are Tesla's (TSLA) peers in the automotive industry along with their respective market capitalizations, listed in descending order of market cap:\n\n- **Tesla, Inc. (TSLA)**\n  - Market Cap: $805.72 billion\n\n- **Ford Motor Company (F)**\n  - Market Cap: $56.54 billion\n\n- **General Motors Company (GM)**\n  - Market Cap: $56.22 billion\n\n- **Rivian Automotive, Inc. (RIVN)**\n  - Market Cap: $17.42 billion\n\n- **NIO Inc. (NIO)**\n  - Market Cap: $9.66 billion"